In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import pandas as pd
import pandas_gbq
import numpy as np
import ast
from google.cloud import storage

In [ ]:
!pip install --upgrade pandas-gbq google-cloud-bigquery
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.34.0
    Uninstalling google-cloud-bigquery-3.34.0:
      Successfully uninstalled google-cloud-bigquery-3.34.0


## 1 Load dữ liệu

In [ ]:

# Load list game
with open("/content/drive/MyDrive/SIC_PROJECT/Data/Raw/games_raw_data.json", "r", encoding="utf-8") as f:
    games_list = json.load(f)

df_list_raw = pd.DataFrame(games_list)

# Load game detail
with open("/content/drive/MyDrive/SIC_PROJECT/Data/Raw/game_details.json", "r", encoding="utf-8") as f:
    details = json.load(f)

df_detail_raw = pd.DataFrame(details)

df_list_raw = df_list_raw[~df_list_raw["id"].isin([51432, 28158])].reset_index(drop=True)
df_detail_raw = df_detail_raw[~df_detail_raw["id"].isin([51432, 28158])].reset_index(drop=True)


In [ ]:
print(df_list_raw.shape)
print(df_list_raw.info())
df_list_raw.head()

(9998, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          9998 non-null   int64  
 1   name        9998 non-null   object 
 2   released    9998 non-null   object 
 3   rating      9998 non-null   float64
 4   metacritic  3974 non-null   float64
 5   playtime    9998 non-null   int64  
 6   genres      9998 non-null   object 
 7   platforms   9998 non-null   object 
 8   added       9998 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 703.1+ KB
None


,id,name,released,rating,metacritic,playtime,genres,platforms,added
0,3498,Grand Theft Auto V,2013-09-17,4.47,92.0,74,[Action],"[PC, PlayStation 5, Xbox One, PlayStation 4, X...",22112
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.64,92.0,43,"[Action, RPG]","[PC, PlayStation 5, Xbox One, PlayStation 4, X...",21768
2,4200,Portal 2,2011-04-18,4.59,95.0,11,"[Shooter, Puzzle]","[PC, Xbox One, macOS, Linux, Xbox 360, PlaySta...",20612
3,4291,Counter-Strike: Global Offensive,2012-08-21,3.57,81.0,64,[Shooter],"[PC, Linux, Xbox 360, PlayStation 3]",18282
4,5286,Tomb Raider (2013),2013-03-05,4.06,86.0,10,[Action],"[PC, Xbox One, PlayStation 4, macOS, Xbox 360,...",17619


In [ ]:
df_list = df_list_raw.drop(columns=["metacritic"])

In [ ]:
print(df_detail_raw.shape)
print(df_detail_raw.info())
df_detail_raw.head()

(9806, 55)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9806 entries, 0 to 9805
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           9806 non-null   int64  
 1   slug                         9806 non-null   object 
 2   name                         9806 non-null   object 
 3   name_original                9806 non-null   object 
 4   description                  9806 non-null   object 
 5   metacritic                   3901 non-null   float64
 6   metacritic_platforms         9806 non-null   object 
 7   released                     9788 non-null   object 
 8   tba                          9806 non-null   bool   
 9   updated                      9806 non-null   object 
 10  background_image             9794 non-null   object 
 11  background_image_additional  9762 non-null   object 
 12  website                      9806 non-null   object 
 13  rating 

,id,slug,name,name_original,description,metacritic,metacritic_platforms,released,tba,updated,...,platforms,stores,developers,genres,tags,publishers,esrb_rating,clip,description_raw,community_rating
0,3498,grand-theft-auto-v,Grand Theft Auto V,Grand Theft Auto V,"<p>Rockstar Games went bigger, since their pre...",92.0,"[{'metascore': 96, 'url': 'https://www.metacri...",2013-09-17,False,2025-07-10T18:19:39,...,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'id': 290376, 'url': '', 'store': {'id': 1, ...","[{'id': 3524, 'name': 'Rockstar North', 'slug'...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[{'id': 2155, 'name': 'Rockstar Games', 'slug'...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",None,"Rockstar Games went bigger, since their previo...",NaN
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,The Witcher 3: Wild Hunt,"<p>The third game in a series, it holds nothin...",92.0,"[{'metascore': 91, 'url': 'https://www.metacri...",2015-05-18,False,2025-07-10T22:38:23,...,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'id': 354780, 'url': '', 'store': {'id': 5, ...","[{'id': 9023, 'name': 'CD PROJEKT RED', 'slug'...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[{'id': 7411, 'name': 'CD PROJEKT RED', 'slug'...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",None,"The third game in a series, it holds nothing b...",NaN
2,4200,portal-2,Portal 2,Portal 2,<p>Portal 2 is a first-person puzzle game deve...,95.0,[],2011-04-18,False,2025-07-09T07:38:04,...,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'id': 465889, 'url': '', 'store': {'id': 2, ...","[{'id': 1612, 'name': 'Valve Software', 'slug'...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[{'id': 354, 'name': 'Electronic Arts', 'slug'...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...",None,Portal 2 is a first-person puzzle game develop...,NaN
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,<p>Counter-Strike is a multiplayer phenomenon ...,81.0,"[{'metascore': 79, 'url': 'https://www.metacri...",2012-08-21,False,2025-07-10T09:42:02,...,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'id': 4619, 'url': '', 'store': {'id': 3, 'n...","[{'id': 1612, 'name': 'Valve Software', 'slug'...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 40847, 'name': 'Steam Achievements', '...","[{'id': 3399, 'name': 'Valve', 'slug': 'valve'...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",None,Counter-Strike is a multiplayer phenomenon in ...,NaN
4,5286,tomb-raider,Tomb Raider (2013),Tomb Raider (2013),<p>A cinematic revival of the series in its ac...,86.0,[],2013-03-05,False,2025-07-05T05:40:57,...,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'id': 33824, 'url': '', 'store': {'id': 7, '...","[{'id': 4037, 'name': 'Crystal Dynamics', 'slu...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[{'id': 308, 'name': 'Square Enix', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",None,A cinematic revival of the series in its actio...,NaN


In [ ]:
valid_ids = set(df_detail_raw["id"])
print(f"Số lượng ID trong file detail: {len(valid_ids)}")

Số lượng ID trong file detail: 9806


In [ ]:
# xóa những recodes trong list không có detail
df_list = df_list[df_list_raw["id"].isin(valid_ids)].reset_index(drop=True)
print(f"Số bản ghi còn lại sau khi lọc: {len(df_list)}")

Số bản ghi còn lại sau khi lọc: 9806


### 2 Loại bỏ các cột không cân thiết

In [ ]:
cols_to_drop = [
    'slug',
    'name_original',
    'description',
    'metacritic_platforms',
    'tba',
    'updated',
    'background_image',
    'background_image_additional',
    'website',
    'reactions',
    'added_by_status',
    'screenshots_count',
    'movies_count',
    'creators_count',
    'achievements_count',
    'parent_achievements_count',
    'reddit_url',
    'reddit_name',
    'reddit_description',
    'reddit_logo',
    'reddit_count',
    'twitch_count',
    'youtube_count',
    'reviews_text_count',
    'suggestions_count',
    'alternative_names',
    'metacritic_url',
    'parents_count',
    'additions_count',
    'game_series_count',
    'user_game',
    'saturated_color',
    'dominant_color',
    'clip',
    'description_raw',
    'stores',
    'developers',
    'publishers',
    'esrb_rating',
    'community_rating',
    'metacritic',
    'parent_platforms'
]
df_detail = df_detail_raw.drop(columns=cols_to_drop)

In [ ]:
df_detail.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9806 entries, 0 to 9805
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             9806 non-null   int64  
 1   name           9806 non-null   object 
 2   released       9788 non-null   object 
 3   rating         9806 non-null   float64
 4   rating_top     9806 non-null   int64  
 5   ratings        9806 non-null   object 
 6   added          9806 non-null   int64  
 7   playtime       9806 non-null   int64  
 8   ratings_count  9806 non-null   int64  
 9   reviews_count  9806 non-null   int64  
 10  platforms      9806 non-null   object 
 11  genres         9806 non-null   object 
 12  tags           9806 non-null   object 
dtypes: float64(1), int64(6), object(6)
memory usage: 996.1+ KB


In [ ]:
df_detail.head()

,id,name,released,rating,rating_top,ratings,added,playtime,ratings_count,reviews_count,platforms,genres,tags
0,3498,Grand Theft Auto V,2013-09-17,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",22112,74,7148,7263,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's..."
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.64,5,"[{'id': 5, 'title': 'exceptional', 'count': 54...",21768,43,6951,7064,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's..."
2,4200,Portal 2,2011-04-18,4.59,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",20612,11,5936,5992,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's..."
3,4291,Counter-Strike: Global Offensive,2012-08-21,3.57,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",18282,64,3586,3624,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 40847, 'name': 'Steam Achievements', '..."
4,5286,Tomb Raider (2013),2013-03-05,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 24...",17619,10,4014,4046,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's..."


## 3 Làm phẳng dữ liệu dạng lồng nhau (Nested Data)

In [ ]:
    """
    Trích xuất tập giá trị duy nhất từ cột nested.

    - df: DataFrame
    - column: tên cột
    - outer_key: key ngoài (ví dụ 'platform')
    - inner_key: key bên trong (ví dụ 'name')

    Cách hoạt động:
    - Nếu outer_key=None và inner_key!=None:
        lấy item[inner_key]
    - Nếu outer_key!=None và inner_key!=None:
        lấy item[outer_key][inner_key]

    Ví dụ:
    - ratings: outer_key=None, inner_key='title'
    - genres: outer_key=None, inner_key='name'
    - platforms: outer_key='platform', inner_key='name'
    """

In [ ]:
def get_unique_nested_values(df, column, outer_key=None, inner_key=None):
    all_items = set()
    for val in df[column].dropna():
        if isinstance(val, list):
            for item in val:
                if isinstance(item, dict):
                    if outer_key is None and inner_key:
                        # Trường hợp phẳng: item['title']
                        if inner_key in item:
                            all_items.add(item[inner_key])
                    elif outer_key and inner_key:
                        # Trường hợp lồng: item['platform']['name']
                        outer = item.get(outer_key)
                        if outer and inner_key in outer:
                            all_items.add(outer[inner_key])

    return all_items

unique_platforms = get_unique_nested_values(
    df_detail,
    column="platforms",
    outer_key="platform",
    inner_key="name"
    )
print("Unique platforms:", unique_platforms)

unique_ratings = get_unique_nested_values(
    df_detail,
    column="ratings",
    outer_key=None,
    inner_key="title"
)
print("Unique ratings:", unique_ratings)

unique_genres = get_unique_nested_values(
    df_detail,
    column="genres",
    outer_key=None,
    inner_key="name"
)
print("Unique genres:", unique_genres)

unique_tags = get_unique_nested_values(
    df_detail,
    column="tags",
    outer_key=None,
    inner_key="name"
)
print("Unique tags:", unique_tags)


Unique platforms: {'Apple II', 'PlayStation 5', 'Linux', 'Xbox One', 'Xbox Series S/X', 'PlayStation 3', 'Web', 'Xbox 360', 'PlayStation', 'PS Vita', 'SEGA Saturn', 'GameCube', 'Nintendo Switch', 'Android', 'PSP', 'PlayStation 4', 'Nintendo DSi', 'Nintendo 3DS', 'Xbox', 'SEGA CD', 'SEGA Master System', 'Genesis', 'Classic Macintosh', 'iOS', 'macOS', 'NES', 'PC', 'PlayStation 2', 'Wii U', 'Game Boy Advance', 'Wii', 'Nintendo DS'}
Unique ratings: {'meh', 'exceptional', 'recommended', 'skip'}
Unique genres: {'Simulation', 'Board Games', 'RPG', 'Action', 'Indie', 'Shooter', 'Fighting', 'Sports', 'Adventure', 'Casual', 'Family', 'Educational', 'Strategy', 'Racing', 'Card', 'Massively Multiplayer', 'Puzzle', 'Arcade', 'Platformer'}
Unique tags: {'race', 'entropy', 'clock', 'Parkour', 'clowns', 'search', 'prehistoric', 'Multiplayer', 'upgrades', 'dice-rolling', 'factions', 'open-ended', 'loss', 'transportation', 'tracks', 'Partial Controller Support', 'big', 'bomb', 'weather', 'Female Protago

Có 4 nested columns

Các cột có nhiều giá trị khác nhau sẽ được làm phẳng thành mảng

Cột ratings sẽ tách ra làm 4 cột khác nhau

In [ ]:
def flatten_list_of_dict(l, key="name"):
    if isinstance(l, list):
        return [d.get(key) for d in l if isinstance(d, dict)]
    return []

def flatten_platforms(l):
    if isinstance(l, list):
        return [d["platform"]["name"] for d in l if "platform" in d]
    return []

# genres
df_detail["genres_list"] = df_detail["genres"].apply(flatten_list_of_dict)

# tags
df_detail["tags_list"] = df_detail["tags"].apply(flatten_list_of_dict)

# platforms
df_detail["platforms_list"] = df_detail["platforms"].apply(flatten_platforms)

# ratings - tạo các cột riêng biệt cho mỗi loại rating
def extract_rating_percent(ratings, title):
    if isinstance(ratings, list):
        for r in ratings:
            if r.get("title") == title:
                return r.get("percent")
    return 0

rating_types = ["exceptional", "recommended", "meh", "skip"]
for rtype in rating_types:
    df_detail[f"rating_{rtype}"] = df_detail["ratings"].apply(lambda x: extract_rating_percent(x, rtype))


In [ ]:
df_detail = df_detail.drop(columns=[
    "ratings", "genres", "tags", "platforms"
])

In [ ]:
df_detail.head()

,id,name,released,rating,rating_top,added,playtime,ratings_count,reviews_count,genres_list,tags_list,platforms_list,rating_exceptional,rating_recommended,rating_meh,rating_skip
0,3498,Grand Theft Auto V,2013-09-17,4.47,5,22112,74,7148,7263,[Action],"[Singleplayer, Steam Achievements, Multiplayer...","[PC, PlayStation 5, Xbox Series S/X, PlayStati...",59.05,32.64,6.37,1.93
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.64,5,21768,43,6951,7064,"[Action, RPG]","[Singleplayer, Full controller support, Atmosp...","[Xbox Series S/X, PlayStation 5, macOS, PlaySt...",76.74,16.32,4.23,2.70
2,4200,Portal 2,2011-04-18,4.59,5,20612,11,5936,5992,"[Shooter, Puzzle]","[Singleplayer, Steam Achievements, Multiplayer...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...",69.24,25.23,3.00,2.52
3,4291,Counter-Strike: Global Offensive,2012-08-21,3.57,4,18282,64,3586,3624,[Shooter],"[Steam Achievements, Multiplayer, Full control...","[PC, Linux, Xbox 360, PlayStation 3]",16.23,46.61,25.94,11.23
4,5286,Tomb Raider (2013),2013-03-05,4.06,4,17619,10,4014,4046,[Action],"[Singleplayer, Multiplayer, Full controller su...","[PlayStation 3, Xbox 360, macOS, PC, Xbox One,...",25.78,60.41,10.85,2.97


## 4 Merge dữ liệu

In [ ]:
df_merged = pd.merge(
    df_list,
    df_detail,
    on="id",
    how="inner",     # chỉ lấy game có cả ở 2 bảng
    suffixes=("_list", "_detail")  # thêm hậu tố để phân biệt cột trùng tên
)
df_merged.head()

,id,name_list,released_list,rating_list,playtime_list,genres,platforms,added_list,name_detail,released_detail,...,playtime_detail,ratings_count,reviews_count,genres_list,tags_list,platforms_list,rating_exceptional,rating_recommended,rating_meh,rating_skip
0,3498,Grand Theft Auto V,2013-09-17,4.47,74,[Action],"[PC, PlayStation 5, Xbox One, PlayStation 4, X...",22112,Grand Theft Auto V,2013-09-17,...,74,7148,7263,[Action],"[Singleplayer, Steam Achievements, Multiplayer...","[PC, PlayStation 5, Xbox Series S/X, PlayStati...",59.05,32.64,6.37,1.93
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.64,43,"[Action, RPG]","[PC, PlayStation 5, Xbox One, PlayStation 4, X...",21768,The Witcher 3: Wild Hunt,2015-05-18,...,43,6951,7064,"[Action, RPG]","[Singleplayer, Full controller support, Atmosp...","[Xbox Series S/X, PlayStation 5, macOS, PlaySt...",76.74,16.32,4.23,2.70
2,4200,Portal 2,2011-04-18,4.59,11,"[Shooter, Puzzle]","[PC, Xbox One, macOS, Linux, Xbox 360, PlaySta...",20612,Portal 2,2011-04-18,...,11,5936,5992,"[Shooter, Puzzle]","[Singleplayer, Steam Achievements, Multiplayer...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...",69.24,25.23,3.00,2.52
3,4291,Counter-Strike: Global Offensive,2012-08-21,3.57,64,[Shooter],"[PC, Linux, Xbox 360, PlayStation 3]",18282,Counter-Strike: Global Offensive,2012-08-21,...,64,3586,3624,[Shooter],"[Steam Achievements, Multiplayer, Full control...","[PC, Linux, Xbox 360, PlayStation 3]",16.23,46.61,25.94,11.23
4,5286,Tomb Raider (2013),2013-03-05,4.06,10,[Action],"[PC, Xbox One, PlayStation 4, macOS, Xbox 360,...",17619,Tomb Raider (2013),2013-03-05,...,10,4014,4046,[Action],"[Singleplayer, Multiplayer, Full controller su...","[PlayStation 3, Xbox 360, macOS, PC, Xbox One,...",25.78,60.41,10.85,2.97


In [ ]:
df_merged.columns

Index(['id', 'name_list', 'released_list', 'rating_list', 'playtime_list',
       'genres', 'platforms', 'added_list', 'name_detail', 'released_detail',
       'rating_detail', 'rating_top', 'added_detail', 'playtime_detail',
       'ratings_count', 'reviews_count', 'genres_list', 'tags_list',
       'platforms_list', 'rating_exceptional', 'rating_recommended',
       'rating_meh', 'rating_skip'],
      dtype='object')

In [ ]:
# Rename cột từ detail
df_merged = df_merged.rename(columns={
    "name_detail": "name",
    "released_detail": "released",
    "rating_detail": "rating",
    "playtime_detail": "playtime",
    "added_detail": "added",
    "tags_list" : "tags"
})

# Drop các cột _list vì đã thay thế:
df_merged = df_merged.drop(columns=[
    "name_list",
    "released_list",
    "rating_list",
    "playtime_list",
    "added_list",
    "genres_list",

    "platforms_list"
])

## 5 Chuẩn hóa dữ liệu

In [ ]:
# Chuẩn hóa date và numeric
df_merged["released"] = pd.to_datetime(df_merged["released"], errors="coerce")
df_merged["rating"] = pd.to_numeric(df_merged["rating"], errors="coerce")
df_merged["playtime"] = pd.to_numeric(df_merged["playtime"], errors="coerce")
df_merged["added"] = pd.to_numeric(df_merged["added"], errors="coerce")

In [ ]:
# Mapping tên game ngày phát hành
release_dates = {
    "Whispers of a Machine": "2019-04-17",
    "Synergy": "2025-04-16",
    "Bless Unleashed": "2021-08-06",
    "Gauntlet": "2014-09-23",
    "Dark and Darker": "2024-06-07",
    "The Wolf Among Us 2": pd.NaT,  # Chưa phát hành chính thức
    "Islands of Insight": "2024-02-13",
    "Secret Neighbor Beta": "2019-08-02",
    "Caliber": "2023-04-12",
    "Halls Of Torment": "2024-09-24",
    "Turbo Golf Racing": "2022-08-04",  # Early Access trên Steam
    "Bloodstained: Curse of the Moon 2": "2020-04-09",
    "The Wandering Village": "2022-02-03",
    "The Last Night": pd.NaT,
    "The Room VR: A Dark Matter": "2020-06-11",
    "Beasts of Maravilla Island": "2022-10-25",
    "Guilty Gear 20th Anniversary Edition": "2023-06-12",
    "Risk of Rain Returns": "2023-02-28",
    "Ravenswatch": "2024-03-05",
    "Len's Island": "2023-04-13"
}

# Cập nhật vào DataFrame
for name, date in release_dates.items():
    df_merged.loc[df_merged["name"] == name, "released"] = date

# định dạng datetime
df_merged["released"] = pd.to_datetime(df_merged["released"], errors="coerce")


In [ ]:
df_merged["year"] = df_merged["released"].dt.year

In [ ]:
df_merged["year"] = df_merged["year"].astype("Int64")

In [ ]:
df_merged['rating'] = df_merged['rating'].replace(0, np.nan)
df_merged['ratings_count'] = df_merged['ratings_count'].replace(0, np.nan)

In [ ]:
df_merged['genres'] = df_merged['genres'].apply(
    lambda x: ['Unknown'] if isinstance(x, list) and len(x)==0 else x
)


In [ ]:
print(df_merged["released"].isna().sum())


0


## 6 Kiểm tra dữ liệu thiếu

In [ ]:
print(df_merged.isna().sum())


id                      0
genres                  0
platforms               0
name                    0
released                0
rating                917
rating_top              0
added                   0
playtime                0
ratings_count          24
reviews_count           0
tags                    0
rating_exceptional      0
rating_recommended      0
rating_meh              0
rating_skip             0
year                    0
dtype: int64


In [ ]:
df_merged.columns

Index(['id', 'genres', 'platforms', 'name', 'released', 'rating', 'rating_top',
       'added', 'playtime', 'ratings_count', 'reviews_count', 'tags',
       'rating_exceptional', 'rating_recommended', 'rating_meh', 'rating_skip',
       'year'],
      dtype='object')

In [ ]:
# Kiểm tra và in ra các dòng có rating là null
print("Các dòng có rating là null:")
print(df_merged[df_merged['rating'].isna()])

# In ra tổng số dòng có rating là null
total_null_ratings = df_merged['rating'].isna().sum()
print(f"\nTổng số dòng có rating là null: {total_null_ratings}")

Các dòng có rating là null:
          id                               genres  \
2628  704730                              [Indie]   
3022  369160                   [Action, Fighting]   
3453  613490                        [Action, RPG]   
3467  301573  [Platformer, Indie, Action, Puzzle]   
3564  427780            [Strategy, Indie, Action]   
...      ...                                  ...   
9797   51221                      [Indie, Action]   
9799   21022                             [Casual]   
9800    8893              [Indie, Arcade, Action]   
9801   21358      [Casual, Adventure, Simulation]   
9802   11550           [Indie, Adventure, Action]   

                               platforms  \
2628                                [PC]   
3022               [PC, Nintendo Switch]   
3453                                [PC]   
3467  [PC, Xbox One, PlayStation 4, Web]   
3564                                [PC]   
...                                  ...   
9797                       

In [ ]:
df_merged.head()

,id,genres,platforms,name,released,rating,rating_top,added,playtime,ratings_count,reviews_count,tags,rating_exceptional,rating_recommended,rating_meh,rating_skip,year
0,3498,[Action],"[PC, PlayStation 5, Xbox One, PlayStation 4, X...",Grand Theft Auto V,2013-09-17,4.47,5,22112,74,7148.0,7263,"[Singleplayer, Steam Achievements, Multiplayer...",59.05,32.64,6.37,1.93,2013
1,3328,"[Action, RPG]","[PC, PlayStation 5, Xbox One, PlayStation 4, X...",The Witcher 3: Wild Hunt,2015-05-18,4.64,5,21768,43,6951.0,7064,"[Singleplayer, Full controller support, Atmosp...",76.74,16.32,4.23,2.70,2015
2,4200,"[Shooter, Puzzle]","[PC, Xbox One, macOS, Linux, Xbox 360, PlaySta...",Portal 2,2011-04-18,4.59,5,20612,11,5936.0,5992,"[Singleplayer, Steam Achievements, Multiplayer...",69.24,25.23,3.00,2.52,2011
3,4291,[Shooter],"[PC, Linux, Xbox 360, PlayStation 3]",Counter-Strike: Global Offensive,2012-08-21,3.57,4,18282,64,3586.0,3624,"[Steam Achievements, Multiplayer, Full control...",16.23,46.61,25.94,11.23,2012
4,5286,[Action],"[PC, Xbox One, PlayStation 4, macOS, Xbox 360,...",Tomb Raider (2013),2013-03-05,4.06,4,17619,10,4014.0,4046,"[Singleplayer, Multiplayer, Full controller su...",25.78,60.41,10.85,2.97,2013


In [ ]:
df_merged[df_merged['genres'].apply(lambda x: 'Unknown' in x)]

,id,genres,platforms,name,released,rating,rating_top,added,playtime,ratings_count,reviews_count,tags,rating_exceptional,rating_recommended,rating_meh,rating_skip,year
561,11148,[Unknown],[PC],The Lab,2016-04-05,3.01,4,3658,1,180.0,182,"[Singleplayer, First-Person, Sci-fi, Funny, St...",9.89,40.66,19.78,29.67,2016
1305,18319,[Unknown],[PC],Painkiller: Black Edition,2007-01-24,3.92,4,1865,2,176.0,177,"[Singleplayer, Atmospheric, Great Soundtrack, ...",25.99,53.11,14.12,6.78,2007
1476,10836,[Unknown],[PC],Source Filmmaker,2012-07-10,3.05,4,1673,1,203.0,203,"[Singleplayer, Multiplayer, Great Soundtrack, ...",11.82,41.87,16.26,30.05,2012
1754,510061,[Unknown],[PC],Ghostrunner Demo,2020-10-22,3.55,4,1432,1,82.0,82,"[combat, Cyberpunk, Story, city, Science, figh...",14.63,52.44,19.51,13.41,2020
1812,3532,[Unknown],"[PlayStation 4, PlayStation 3]",SingStar,2014-10-28,2.45,1,1386,0,216.0,216,[exclusive],4.17,18.98,35.65,41.20,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9567,60729,[Unknown],[PC],Vカツ / V-Katsu,2018-08-01,2.67,4,168,1,6.0,6,"[Singleplayer, Partial Controller Support, Fre...",0.00,33.33,33.33,33.33,2018
9659,20144,[Unknown],[PC],RPG Maker 2003,2015-04-24,4.08,4,166,1,12.0,12,"[RPG, RPGMaker, Design & Illustration, Game De...",33.33,58.33,0.00,8.33,2015
9698,28352,[Unknown],[Xbox 360],Mars Rover Landing,2012-06-16,2.37,3,165,0,19.0,19,"[achievements, Mars]",0.00,10.53,52.63,36.84,2012
9753,13509,[Unknown],[PC],SLIVER.tv,2016-09-20,NaN,0,164,1,2.0,2,"[Early Access, Utilities, Video Production]",0.00,0.00,0.00,100.00,2016


In [ ]:
# Đếm số lượng dòng có ratings_count=0 và rating=0
no_ratings_count = df_merged[(df_merged['ratings_count'] == 0) & (df_merged['rating'] == 0)].shape[0]
total_rows = df_merged.shape[0]
no_ratings_percentage = (no_ratings_count / total_rows) * 100

print(f"Số dòng ratings_count=0 & rating=0: {no_ratings_count}")
print(f"Tỷ lệ: {no_ratings_percentage:.2f}%")

# Đếm số lượng dòng có genres rỗng ([])
empty_genres_count = df_merged[df_merged['genres'].apply(lambda x: isinstance(x, list) and len(x) == 0)].shape[0]
empty_genres_percentage = (empty_genres_count / total_rows) * 100

print(f"Số dòng genres=[]: {empty_genres_count}")
print(f"Tỷ lệ: {empty_genres_percentage:.2f}%")


Số dòng ratings_count=0 & rating=0: 0
Tỷ lệ: 0.00%
Số dòng genres=[]: 0
Tỷ lệ: 0.00%


In [ ]:
df_merged[df_merged['ratings_count']==0]

,id,genres,platforms,name,released,rating,rating_top,added,playtime,ratings_count,reviews_count,tags,rating_exceptional,rating_recommended,rating_meh,rating_skip,year


In [ ]:
df_merged[df_merged["rating"] == 0].sort_values("ratings_count").tail()


,id,genres,platforms,name,released,rating,rating_top,added,playtime,ratings_count,reviews_count,tags,rating_exceptional,rating_recommended,rating_meh,rating_skip,year


In [ ]:
# Đảm bảo index sạch
df_save = df_merged.reset_index(drop=True)

df_save.to_csv("/content/drive/MyDrive/SIC_PROJECT/Data/Processed/cleaned_games_raw.csv", index=False)


# Lưu Parquet
df_save.to_parquet("/content/drive/MyDrive/SIC_PROJECT/Data/Processed/cleaned_games_raw.parquet", index=False)


# Push dữ liệu lên datawarehouse

In [ ]:
key_path = "YOUR_KEY_PATH"

In [ ]:
creds = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=creds, project="crawlgame")

In [ ]:
pandas_gbq.to_gbq(
    df_save,
    destination_table='game_data.cleaned_games',  # <dataset>.<table>
    project_id='crawlgame',
    credentials=creds,
    if_exists='replace'
)

100%|██████████| 1/1 [00:00<00:00, 8886.24it/s]
